# In dit project wil ik Bitcoin onderzoeken en ook de prijs van Bitcoin voorspellen met behulp van een Machine Learning algoritme.


In [ ]:
# In dit project wil ik Bitcoin onderzoeken en ook de prijs van Bitcoin voorspellen met behulp van een Machine Learning algoritme.

# Hier importeer ik de benodigde Bibliotheek
import os
import pandas as pd
import numpy as np
import math
import datetime as dt
import matplotlib.pyplot as plt

# Bibioliotheek voor de evaluatie 
from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score 
from sklearn.metrics import mean_poisson_deviance, mean_gamma_deviance, accuracy_score
from sklearn.preprocessing import MinMaxScaler

# Bibioliotheek voor het bouwen van de model 
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import LSTM

# Bibioliotheek voor het plotten 
import matplotlib.pyplot as plt
from itertools import cycle
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

In [ ]:
# Bitcoin dataset van Yahoo. 2014 sept t/m 2022 nov.
maindf=pd.read_csv("bitcoin(update).csv")

In [ ]:
# Hier print ik totaal aantal dagen (2974) en de kolommen (7)
print('Totaal aantal dagen in de dataset: ',maindf.shape[0])
print('Totaal aantal kolommen in de dataset: ',maindf.shape[1])

In [ ]:
maindf.shape

##  Description
| Name       | Description                                                                                      |
| ------------------ | ------------------------------------------------------------------------------------------------ |
|  |
| Date            |  De datum van de dag                                                                                   |
| Open  | Openingskoers van bitcoin                                                       |
| High  | Wat de hoogste waarde is van de desbetreffende dag                                                         |
| Low    |  Wat de laagste waarde is van de desbetreffende dag                                                     |
| Close    | De sloetkoers van bitcoin                                                      |
| Adj Close, Volume    | Deze laat ik weg. Hier doe ik niks mee                                                   |

De eerste 5 en rows van de dataset

In [ ]:
maindf.head()

De laatste 5 en rows van de dataset

In [ ]:
maindf.tail()

Algemene info. Wat zijn de aanwezige kolommen, null type en data type

In [ ]:
maindf.info()

#### De beschrijving voor elke kolom:
    - count - Het aantal niet-lege waarden.
    - mean - De gemiddelde waarde.
    - std - De standaardafwijking.
    - min - De minimumwaarde.
    - 25% - Het 25%-percentiel*.
    - 50% - Het 50% percentiel*.
    - 75% - het 75%-percentiel*.
    - max - de maximumwaarde.

In [ ]:
maindf.describe()

# 1. Checken voor Null values

In [ ]:
# Deze 2 stappen zijn een van de belangrijkste dingen om mee te beginnen. Ik Check of de data nulwaarden heeft of niet. 
print('Null Values:',maindf.isnull().values.sum())

In [ ]:
print('NA values:',maindf.isnull().values.any())

In [ ]:
print(f'Number of rows with missing values: {maindf.isnull().any(axis=1).mean()}')

#### De data heeft geen null values. 

# 2. Exploratory Data Analysis (EDA) 

Ik print de start en de einddatum uit

In [ ]:
start=maindf.iloc[0][0]
eind=maindf.iloc[-1][0]

print('begindatum', start)
print('einddatum', eind)

In [ ]:
maindf.info()

Ik converteer ik de datum van de dataframe naar de datum tijd. Daarna maak ik de kolommen voor: maand, dag en jaar.

In [ ]:
maindf['Date'] = pd.to_datetime(maindf['Date'], format='%Y-%m-%d')
maindf['Dag'] = maindf['Date'].dt.day
maindf['Maand'] = maindf['Date'].dt.month
maindf['Jaar'] = maindf['Date'].dt.year

# 2.1 Analyse jaar: 2014

Ik maak een variabele aan met de naam: jaar_2014 die alleen de gegevens van jaar_2014 zal bevatten. De kolommen 'Adj Close' en 'Volume' laat ik weg

In [ ]:
jaar_2014 = maindf.loc[(maindf['Date'] >= '2014') 
                     & (maindf['Date'] < '2015')]

jaar_2014.drop(jaar_2014[['Adj Close','Volume']],axis=1)

De gemiddelde waarde per maand weergeven. Jan t/m aug is er geen data. Dit komt omdat de begindatum 17-09 is.

In [ ]:
gemiddelde= jaar_2014.groupby(jaar_2014['Date'].dt.strftime('%B'))[['Open','Close']].mean()
maanden = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 
             'September', 'October', 'November', 'December']
gemiddelde = gemiddelde.reindex(maanden, axis=0)
gemiddelde

Een barchart die laat zien wat de opening prijs en de gesloten prijs is van het gemiddelde van elke maand. Geel: open, Groen: gesloten

In [ ]:
fig = go.Figure()

fig.add_trace(go.Bar(
    x=gemiddelde.index,
    y=gemiddelde['Open'],
    name='Bitcoin open prijs',
    marker_color='Yellow'
))
fig.add_trace(go.Bar(
    x=gemiddelde.index,
    y=gemiddelde['Close'],
    name='Bitcoin gelosten prijs',
    marker_color='Green'
))

fig.update_layout(barmode='group', xaxis_tickangle=-45, 
                  title='Maandelijkse vergelijking tussen open en gesloten prijzen')
fig.show()

Een barchart die laat zien wat de hoge prijs en de lage prijs is van het gemiddelde van elke maand

In [ ]:
fig = go.Figure()

jaar_2014.groupby(jaar_2014['Date'].dt.strftime('%B'))['Low'].min()
gemiddelde_high = jaar_2014.groupby(maindf['Date'].dt.strftime('%B'))['High'].max()
gemiddelde_high = gemiddelde_high.reindex(maanden, axis=0)

gemiddelde_low = jaar_2014.groupby(jaar_2014['Date'].dt.strftime('%B'))['Low'].min()
gemiddelde_low = gemiddelde_low.reindex(maanden, axis=0)

fig = go.Figure()
fig.add_trace(go.Bar(
    x=gemiddelde_high.index,
    y=gemiddelde_high,
    name='Bitcoin hoog prijs',
    marker_color='Blue'
))
fig.add_trace(go.Bar(
    x=gemiddelde_low.index,
    y=gemiddelde_low,
    name='Bitcoin laag prijs',
    marker_color='Red'
))

fig.update_layout(barmode='group', 
                  title='Maandelijkse vergelijking tussen Hoge en Lage prijzen:')
fig.show()

Een boxplot die de maandelijkse gesloten prijs laat zien van het jaar 2014

In [ ]:
fig = px.box(jaar_2014,
             x="Maand",
             y="Close",
             labels = {"Close" : "Gesloten prijs"}, 
             title="Maandelijkse vergelijking gesloten prijs")
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

Een boxplot die de gesloten prijs voor elke dag van elke maand laat zien van het jaar 2014.
Dus van alle 30 of 31 dagen de gesloten prijs

In [ ]:
fig = px.box(jaar_2014,
             x="Dag",
             y="Close",
             labels = {"Close" : "Gesloten prijs"}, 
             title="Gesloten prijs voor elke dag van 12 maanden")
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

Een strip plot die de gesloten prijs voor elke dag van van elke maand laat zien van het jaar 2014.

In [ ]:
fig = px.strip(jaar_2014,
               x="Dag",
               y="Close",
               labels = {"Close" : "Gesloten prijs"},
               title = "Gesloten prijs voor elke dag van 12 maanden")
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()


Een strip plot die de gesloten prijs voor elke dag van van elke maand laat zien van het jaar 2014. Als extra functie kan je maanden apart bekijken en vergelijken.

In [ ]:
fig = px.strip(jaar_2014,
               x="Dag",
               y="Close",
               labels = {"Close" : "Gesloten prijs"},
               color="Maand",
               title = "Gesloten prijs voor elke dag van 12 maanden"
              )
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

Een line chart die een samenvatting van open, gelosten, hoog en laag laat zien wat de algemene trend van het jaar 2014 was

In [ ]:
names = cycle(['Bitcoin Open Prijs','Bitcoin Gesloten Prijs','Bitcoin Hoog Prijs','Bitcoin Laag Prijs'])

fig = px.line(jaar_2014, x=jaar_2014.Date, y=[jaar_2014['Open'], jaar_2014['Close'], 
                                          jaar_2014['High'], jaar_2014['Low']],
             labels={'Date': 'Datum','value':'Waarde Bitcoin'})
fig.update_layout(title_text='Grafiek Bitcoin analyse', font_size=15, font_color='black',legend_title_text='Bitcoin Parameters:')
fig.for_each_trace(lambda t:  t.update(name = next(names)))
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)


# 2.2 Analyse jaar: 2015 

Ik maak een variabele aan met de naam: jaar_2015 die alleen de gegevens van jaar_2015 zal bevatten. De kolommen 'Adj Close' en 'Volume' laat ik weg

In [ ]:
jaar_2015 = maindf.loc[(maindf['Date'] >= '2015')
                     & (maindf['Date'] < '2016')]

jaar_2015.drop(jaar_2015[['Adj Close','Volume']],axis=1)

De gemiddelde waarde per maand weergeven.

In [ ]:
gemiddelde= jaar_2015.groupby(jaar_2015['Date'].dt.strftime('%B'))[['Open','Close']].mean()
maanden = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 
             'September', 'October', 'November', 'December']
gemiddelde = gemiddelde.reindex(maanden, axis=0)
gemiddelde

Een barchart die laat zien wat de opening prijs en de gesloten prijs is van het gemiddelde van elke maand. Geel: open, Groen: gesloten

In [ ]:
fig = go.Figure()

fig.add_trace(go.Bar(
    x=gemiddelde.index,
    y=gemiddelde['Open'],
    name='Bitcoin open prijs',
    marker_color='Yellow'
))
fig.add_trace(go.Bar(
    x=gemiddelde.index,
    y=gemiddelde['Close'],
    name='Bitcoin gesloten prijs',
    marker_color='Green'
))

fig.update_layout(barmode='group', xaxis_tickangle=-45, 
                  title='Maandelijkse vergelijking tussen open en gesloten prijzen')
fig.show()

Een barchart die laat zien wat de hoge prijs en de lage prijs is van het gemiddelde van elke maand

In [ ]:
jaar_2015.groupby(jaar_2015['Date'].dt.strftime('%B'))['Low'].min()
gemiddelde_high = jaar_2015.groupby(maindf['Date'].dt.strftime('%B'))['High'].max()
gemiddelde_high = gemiddelde_high.reindex(maanden, axis=0)

gemiddelde_low = jaar_2015.groupby(jaar_2015['Date'].dt.strftime('%B'))['Low'].min()
gemiddelde_low = gemiddelde_low.reindex(maanden, axis=0)

fig = go.Figure()
fig.add_trace(go.Bar(
    x=gemiddelde_high.index,
    y=gemiddelde_high,
    name='Hoog Bitcoin prijs',
    marker_color='Blue'
))
fig.add_trace(go.Bar(
    x=gemiddelde_low.index,
    y=gemiddelde_low,
    name='Laag Bitcoin prijs',
    marker_color='Red'
))

fig.update_layout(barmode='group', 
                  title='Maandelijkse vergelijking tussen Hoge en Lage prijzen')
fig.show()

Een boxplot die de maandelijkse gesloten prijs laat zien van het jaar 2015

In [ ]:
fig = px.box(jaar_2015,
             x="Maand",
             y="Close",
             labels = {"Close" : "Gesloten prijs"}, # change label of y-axis
             title="Maandelijkse vergelijking gesloten prijs")
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

Een boxplot die de gesloten prijs voor elke dag van elke maand laat zien van het jaar 2015.

Dus van alle 30 of 31 dagen de gesloten prijs.

In [ ]:
fig = px.box(jaar_2015,
             x="Dag",
             y="Close",
             labels = {"Close" : "Gesloten prijs"}, # change label of y-axis
             title="Gesloten prijs voor elke dag van 12 maanden")
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

Een strip plot die de gesloten prijs voor elke dag van van elke maand laat zien van het jaar 2015.

In [ ]:
fig = px.strip(jaar_2015,
               x="Dag",
               y="Close",
               labels = {"Close" : "Gesloten prijs"},
               title = "Gesloten prijs voor elke dag van 12 maanden")
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

Een strip plot die de gesloten prijs voor elke dag van van elke maand laat zien van het jaar 2015. Als extra functie kan je maanden apart bekijken en vergelijken.

In [ ]:
fig = px.strip(jaar_2015,
               x="Dag",
               y="Close",
               labels = {"Close" : "Gesloten prijs"},
               color="Maand",
               title = "Strip plot"
              )
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

Een line chart die een samenvatting van open, gelosten, hoog en laag laat zien wat de algemene trend van het jaar 2015 was

In [ ]:
names = cycle(['Bitcoin open prijs','Bitcoin gesloten prijs','Bitcoin hoog prijs','Bitcoin laag prijs'])

fig = px.line(jaar_2015, x=jaar_2015.Date, y=[jaar_2015['Open'], jaar_2015['Close'], 
                                          jaar_2015['High'], jaar_2015['Low']],
             labels={'Date': 'Datum','value':'Waarde Bitcoin'})
fig.update_layout(title_text='Grafiek Bitcoin analyse', font_size=15, font_color='black',legend_title_text='Aandeel Bitcoin:')
fig.for_each_trace(lambda t:  t.update(name = next(names)))
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)

fig.show()

# 2.3 Analyse jaar: 2016 

Ik maak een variabele aan met de naam: jaar_2016 die alleen de gegevens van jaar_2016 zal bevatten. De kolommen 'Adj Close' en 'Volume' laat ik weg

In [ ]:
jaar_2016 = maindf.loc[(maindf['Date'] >= '2016')
                     & (maindf['Date'] < '2017')]

jaar_2016.drop(jaar_2016[['Adj Close','Volume']],axis=1)

De gemiddelde waarde per maand weergeven.

In [ ]:
gemiddelde= jaar_2016.groupby(jaar_2016['Date'].dt.strftime('%B'))[['Open','Close']].mean()
maanden = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 
             'September', 'October', 'November', 'December']
dagen = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
gemiddelde = gemiddelde.reindex(maanden, axis=0)
gemiddelde

Een barchart die laat zien wat de opening prijs en de gesloten prijs is van het gemiddelde van elke maand. Geel: open, Groen: gesloten

In [ ]:
fig = go.Figure()

fig.add_trace(go.Bar(
    x=gemiddelde.index,
    y=gemiddelde['Open'],
    name='Bitcoin open prijs',
    marker_color='Yellow'
))
fig.add_trace(go.Bar(
    x=gemiddelde.index,
    y=gemiddelde['Close'],
    name='Bitcoin gesloten prijs',
    marker_color='Green'
))

fig.update_layout(barmode='group', xaxis_tickangle=-45, 
                  title='Maandelijkse vergelijking tussen open en gesloten prijzen')
fig.show()

Een barchart die laat zien wat de hoge prijs en de lage prijs is van het gemiddelde van elke maand

In [ ]:
jaar_2016.groupby(jaar_2016['Date'].dt.strftime('%B'))['Low'].min()
gemiddelde_high = jaar_2016.groupby(maindf['Date'].dt.strftime('%B'))['High'].max()
gemiddelde_high = gemiddelde_high.reindex(maanden, axis=0)

gemiddelde_low = jaar_2016.groupby(jaar_2016['Date'].dt.strftime('%B'))['Low'].min()
gemiddelde_low = gemiddelde_low.reindex(maanden, axis=0)

fig = go.Figure()
fig.add_trace(go.Bar(
    x=gemiddelde_high.index,
    y=gemiddelde_high,
    name='Bitcoin hoog prijs',
    marker_color='Blue'
))
fig.add_trace(go.Bar(
    x=gemiddelde_low.index,
    y=gemiddelde_low,
    name='Bitcoin laag prijs',
    marker_color='Red'
))

fig.update_layout(barmode='group', 
                  title='Maandelijkse vergelijking tussen Hoge en Lage prijzen')
fig.show()

Een boxplot die de maandelijkse gesloten prijs laat zien van het jaar 2016.

In [ ]:
fig = px.box(jaar_2016,
             x="Maand",
             y="Close",
             labels = {"Close" : "Gesloten prijs"}, # change label of y-axis
             title="Maandelijkse vergelijking gesloten prijs")
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

Een boxplot die de gesloten prijs voor elke dag van elke maand laat zien van het jaar 2016.
Dus van alle 30 of 31 dagen de gesloten prijs

In [ ]:
fig = px.box(jaar_2016,
             x="Dag",
             y="Close",
             labels = {"Close" : "Gesloten prijs"}, # change label of y-axis
             title="Gesloten prijs voor elke dag van 12 maanden")
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

Een strip plot die de gesloten prijs voor elke dag van van elke maand laat zien van het jaar 2016.

In [ ]:
fig = px.strip(jaar_2016,
               x="Dag",
               y="Close",
               labels = {"Close" : "Gesloten prijs"},
               title = "Gesloten prijs voor elke dag van 12 maanden")
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

Een strip plot die de gesloten prijs voor elke dag van van elke maand laat zien van het jaar 2016. Als extra functie kan je maanden apart bekijken en vergelijken.

In [ ]:
fig = px.strip(jaar_2016,
               x="Dag",
               y="Close",
               labels = {"Close" : "Gesloten prijs"},
               color="Maand",
               title = "Strip plot"
              )
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

Een line chart die een samenvatting van open, gelosten, hoog en laag laat zien wat de algemene trend van het jaar 2016 was

In [ ]:
names = cycle(['Bitcoin open prijs','Bitcoin gesloten prijs','Bitcoin hoog prijs','Bitcoin laag prijs'])

fig = px.line(jaar_2016, x=jaar_2016.Date, y=[jaar_2016['Open'], jaar_2016['Close'], 
                                          jaar_2016['High'], jaar_2016['Low']],
             labels={'Date': 'Datum','value':'Bitcoin aandeel'})
fig.update_layout(title_text='Grafiek Bitcoin analyse', font_size=15, font_color='black',legend_title_text='Bitcoin Parameters:')
fig.for_each_trace(lambda t:  t.update(name = next(names)))
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)

fig.show()

# 2.4 Analyse jaar: 2017 

Ik maak een variabele aan met de naam: jaar_2017 die alleen de gegevens van jaar_2017 zal bevatten. De kolommen 'Adj Close' en 'Volume' laat ik weg

In [ ]:
jaar_2017 = maindf.loc[(maindf['Date'] >= '2017')
                     & (maindf['Date'] < '2018')]

jaar_2017.drop(jaar_2017[['Adj Close','Volume']],axis=1)

De gemiddelde waarde per maand weergeven. 

In [ ]:
gemiddelde= jaar_2017.groupby(jaar_2017['Date'].dt.strftime('%B'))[['Open','Close']].mean()
maanden = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 
             'September', 'October', 'November', 'December']
gemiddelde = gemiddelde.reindex(maanden, axis=0)
gemiddelde

Een barchart die laat zien wat de opening prijs en de gesloten prijs is van het gemiddelde van elke maand. Geel: open, Groen: gesloten

In [ ]:
fig = go.Figure()

fig.add_trace(go.Bar(
    x=gemiddelde.index,
    y=gemiddelde['Open'],
    name='Bitcoin open prijs',
    marker_color='Yellow'
))
fig.add_trace(go.Bar(
    x=gemiddelde.index,
    y=gemiddelde['Close'],
    name='Bitcoin gesloten prijs',
    marker_color='Green'
))

fig.update_layout(barmode='group', xaxis_tickangle=-45, 
                  title='Maandelijkse vergelijking tussen open en gesloten prijzen')
fig.show()

Een barchart die laat zien wat de hoge prijs en de lage prijs is van het gemiddelde van elke maand

In [ ]:
jaar_2017.groupby(jaar_2017['Date'].dt.strftime('%B'))['Low'].min()
gemiddelde_high = jaar_2017.groupby(maindf['Date'].dt.strftime('%B'))['High'].max()
gemiddelde_high = gemiddelde_high.reindex(maanden, axis=0)

gemiddelde_low = jaar_2017.groupby(jaar_2017['Date'].dt.strftime('%B'))['Low'].min()
gemiddelde_low = gemiddelde_low.reindex(maanden, axis=0)

fig = go.Figure()
fig.add_trace(go.Bar(
    x=gemiddelde_high.index,
    y=gemiddelde_high,
    name='Bitcoin hoog prijs',
    marker_color='Blue'
))
fig.add_trace(go.Bar(
    x=gemiddelde_low.index,
    y=gemiddelde_low,
    name='Bitcoin laag prijs',
    marker_color='Red'
))

fig.update_layout(barmode='group', 
                  title='Maandelijkse vergelijking tussen Hoge en Lage prijzen')
fig.show()

Een boxplot die de maandelijkse gesloten prijs laat zien van het jaar 2017

In [ ]:
fig = px.box(jaar_2017,
             x="Maand",
             y="Close",
             labels = {"Close" : "Gesloten prijs"}, # change label of y-axis
             title="Maandelijkse vergelijking gesloten prijs")
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

Een boxplot die de gesloten prijs voor elke dag van elke maand laat zien van het jaar 2017.
Dus van alle 30 of 31 dagen de gesloten prijs

In [ ]:
fig = px.box(jaar_2017,
             x="Dag",
             y="Close",
             labels = {"Close" : "Gesloten prijs"}, # change label of y-axis
             title="Gesloten prijs voor elke dag van 12 maanden")
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

Een strip plot die de gesloten prijs voor elke dag van van elke maand laat zien van het jaar 2017.

In [ ]:
fig = px.strip(jaar_2017,
               x="Dag",
               y="Close",
               labels = {"Close" : "Gesloten prijs"},
               title = "Gesloten prijs voor elke dag van 12 maanden")
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

Een strip plot die de gesloten prijs voor elke dag van van elke maand laat zien van het jaar 2017. Als extra functie kan je maanden apart bekijken en vergelijken.

In [ ]:
fig = px.strip(jaar_2017,
               x="Dag",
               y="Close",
               labels = {"Close" : "Gesloten prijs"},
               color="Maand",
               title = "Strip plot"
              )
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

Een line chart die een samenvatting van open, gelosten, hoog en laag laat zien wat de algemene trend van het jaar 2017 was

In [ ]:
names = cycle(['Bitcoin open prijs','Bitcoin gesloten prijs','Bitcoin hoog prijs','Bitcoin laag prijs'])

fig = px.line(jaar_2017, x=jaar_2017.Date, y=[jaar_2017['Open'], jaar_2017['Close'], 
                                          jaar_2017['High'], jaar_2017['Low']],
             labels={'Date': 'Datum','value':'Waarde Bitcoin'})
fig.update_layout(title_text='Grafiek Bitcoin analyse', font_size=15, font_color='black',legend_title_text='Parameters Bitcoin:')
fig.for_each_trace(lambda t:  t.update(name = next(names)))
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)

fig.show()

#  2.5 Analyse jaar: 2018

Ik maak een variabele aan met de naam: jaar_2018 die alleen de gegevens van jaar_2018 zal bevatten. De kolommen 'Adj Close' en 'Volume' laat ik weg

In [ ]:
jaar_2018 = maindf.loc[(maindf['Date'] >= '2018')
                     & (maindf['Date'] < '2019')]

jaar_2018.drop(jaar_2018[['Adj Close','Volume']],axis=1)

De gemiddelde waarde per maand weergeven

In [ ]:
gemiddelde= jaar_2018.groupby(jaar_2018['Date'].dt.strftime('%B'))[['Open','Close']].mean()
maanden = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 
             'September', 'October', 'November', 'December']
gemiddelde = gemiddelde.reindex(maanden, axis=0)
gemiddelde

Een barchart die laat zien wat de opening prijs en de gesloten prijs is van het gemiddelde van elke maand. Geel: open, Groen: gesloten

In [ ]:
fig = go.Figure()

fig.add_trace(go.Bar(
    x=gemiddelde.index,
    y=gemiddelde['Open'],
    name='Bitcoin open prijs',
    marker_color='Yellow'
))
fig.add_trace(go.Bar(
    x=gemiddelde.index,
    y=gemiddelde['Close'],
    name='Bitcoin gesloten prijs',
    marker_color='Green'
))

fig.update_layout(barmode='group', xaxis_tickangle=-45, 
                  title='Maandelijkse vergelijking tussen open en gesloten prijzen')
fig.show()

Een barchart die laat zien wat de hoge prijs en de lage prijs is van het gemiddelde van elke maand

In [ ]:
jaar_2018.groupby(jaar_2018['Date'].dt.strftime('%B'))['Low'].min()
gemiddelde_high = jaar_2018.groupby(maindf['Date'].dt.strftime('%B'))['High'].max()
gemiddelde_high = gemiddelde_high.reindex(maanden, axis=0)

gemiddelde_low = jaar_2018.groupby(jaar_2018['Date'].dt.strftime('%B'))['Low'].min()
gemiddelde_low = gemiddelde_low.reindex(maanden, axis=0)

fig = go.Figure()
fig.add_trace(go.Bar(
    x=gemiddelde_high.index,
    y=gemiddelde_high,
    name='Bitcoin hoog prijs',
    marker_color='Blue'
))
fig.add_trace(go.Bar(
    x=gemiddelde_low.index,
    y=gemiddelde_low,
    name='Bitcoin laag prijs',
    marker_color='Red'
))

fig.update_layout(barmode='group', 
                  title='Maandelijkse vergelijking tussen Hoge en Lage prijzen')
fig.show()

Een boxplot die de maandelijkse gesloten prijs laat zien van het jaar 2018

In [ ]:
fig = px.box(jaar_2018,
             x="Maand",
             y="Close",
             labels = {"Close" : "Gesloten prijs"}, # change label of y-axis
             title="Maandelijkse vergelijking gesloten prijs")
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

Een boxplot die de gesloten prijs voor elke dag van elke maand laat zien van het jaar 2018.
Dus van alle 30 of 31 dagen de gesloten prijs

In [ ]:
fig = px.box(jaar_2018,
             x="Dag",
             y="Close",
             labels = {"Close" : "Gesloten prijs"}, # change label of y-axis
             title="Gesloten prijs voor elke dag van 12 maanden")
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

Een strip plot die de gesloten prijs voor elke dag van van elke maand laat zien van het jaar 2018.

In [ ]:
fig = px.strip(jaar_2018,
               x="Dag",
               y="Close",
               labels = {"Close" : "Gesloten prijs"},
               title = "Gesloten prijs voor elke dag van 12 maanden")
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

Een strip plot die de gesloten prijs voor elke dag van van elke maand laat zien van het jaar 2018. Als extra functie kan je maanden apart bekijken en vergelijken.

In [ ]:
fig = px.strip(jaar_2018,
               x="Dag",
               y="Close",
               labels = {"Close" : "Gesloten prijs"},
               color="Maand",
               title = "Gesloten prijs voor elke dag van 12 maanden"
              )
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

Een line chart die een samenvatting van open, gelosten, hoog en laag laat zien wat de algemene trend van het jaar 2018 was

In [ ]:
names = cycle(['Bitcoin open prijs','Bitcoin gesloten prijs','Bitcoin hoog prijs','Bitcoin laag prijs'])

fig = px.line(jaar_2018, x=jaar_2018.Date, y=[jaar_2018['Open'], jaar_2018['Close'], 
                                          jaar_2018['High'], jaar_2018['Low']],
             labels={'Date': 'Datum','value':'Waarde Bitcoin'})
fig.update_layout(title_text='Grafiek Bitcoin analyse', font_size=15, font_color='black',legend_title_text='Bitcoin Parameters:')
fig.for_each_trace(lambda t:  t.update(name = next(names)))
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)

fig.show()

#  2.6 Analyse jaar: 2019

Ik maak een variabele aan met de naam: jaar_2019 die alleen de gegevens van jaar_2019 zal bevatten. De kolommen 'Adj Close' en 'Volume' laat ik weg

In [ ]:
jaar_2019 = maindf.loc[(maindf['Date'] >= '2019')
                     & (maindf['Date'] < '2020')]

jaar_2019.drop(jaar_2019[['Adj Close','Volume']],axis=1)

De gemiddelde waarde per maand weergeven

In [ ]:
gemiddelde= jaar_2019.groupby(jaar_2019['Date'].dt.strftime('%B'))[['Open','Close']].mean()
maanden = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 
             'September', 'October', 'November', 'December']
gemiddelde = gemiddelde.reindex(maanden, axis=0)
gemiddelde

Een barchart die laat zien wat de opening prijs en de gesloten prijs is van het gemiddelde van elke maand. Geel: open, Groen: gesloten

In [ ]:
fig = go.Figure()

fig.add_trace(go.Bar(
    x=gemiddelde.index,
    y=gemiddelde['Open'],
    name='Bitcoin open prijs',
    marker_color='Yellow'
))
fig.add_trace(go.Bar(
    x=gemiddelde.index,
    y=gemiddelde['Close'],
    name='Bitcoin gesloten prijs',
    marker_color='Green'
))

fig.update_layout(barmode='group', xaxis_tickangle=-45, 
                  title='Maandelijkse vergelijking tussen open en gesloten prijzen')
fig.show()

Een barchart die laat zien wat de hoge prijs en de lage prijs is van het gemiddelde van elke maand

In [ ]:
jaar_2019.groupby(jaar_2019['Date'].dt.strftime('%B'))['Low'].min()
gemiddelde_high = jaar_2019.groupby(maindf['Date'].dt.strftime('%B'))['High'].max()
gemiddelde_high = gemiddelde_high.reindex(maanden, axis=0)

gemiddelde_low = jaar_2019.groupby(jaar_2019['Date'].dt.strftime('%B'))['Low'].min()
gemiddelde_low = gemiddelde_low.reindex(maanden, axis=0)

fig = go.Figure()
fig.add_trace(go.Bar(
    x=gemiddelde_high.index,
    y=gemiddelde_high,
    name='Bitcoin hoog prijs',
    marker_color='Blue'
))
fig.add_trace(go.Bar(
    x=gemiddelde_low.index,
    y=gemiddelde_low,
    name='Bitcoin laag prijs',
    marker_color='Red'
))

fig.update_layout(barmode='group', 
                  title='Maandelijkse vergelijking tussen Hoge en Lage prijzen')
fig.show()

Een boxplot die de maandelijkse gesloten prijs laat zien van het jaar 2019.

In [ ]:
fig = px.box(jaar_2019,
             x="Maand",
             y="Close",
             labels = {"Close" : "Gesloten prijs"}, # change label of y-axis
             title="Maandelijkse vergelijking gesloten prijs")
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

Een boxplot die de gesloten prijs voor elke dag van elke maand laat zien van het jaar 2019.
Dus van alle 30 of 31 dagen de gesloten prijs.

In [ ]:
fig = px.box(jaar_2019,
             x="Dag",
             y="Close",
             labels = {"Close" : "Gesloten prijs"}, # change label of y-axis
             title="Gesloten prijs voor elke dag van 12 maanden")
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

Een strip plot die de gesloten prijs voor elke dag van van elke maand laat zien van het jaar 2019.

In [ ]:
fig = px.strip(jaar_2019,
               x="Dag",
               y="Close",
               labels = {"Close" : "Gesloten prijs"},
               title = "Gesloten prijs voor elke dag van 12 maanden")
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

Een strip plot die de gesloten prijs voor elke dag van van elke maand laat zien van het jaar 2019. Als extra functie kan je maanden apart bekijken en vergelijken.

In [ ]:
fig = px.strip(jaar_2019,
               x="Dag",
               y="Close",
               labels = {"Close" : "Gesloten prijs"},
               color="Maand",
               title = "Gesloten prijs voor elke dag van 12 maanden"
              )
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

Een line chart die een samenvatting van open, gelosten, hoog en laag laat zien wat de algemene trend van het jaar 2019 was

In [ ]:
names = cycle(['Bitcoin open prijs','Bitcoin gesloten prijs','Bitcoin hoog prijs','Bitcoin laag prijs'])

fig = px.line(jaar_2019, x=jaar_2019.Date, y=[jaar_2019['Open'], jaar_2019['Close'], 
                                          jaar_2019['High'], jaar_2019['Low']],
             labels={'Date': 'Datum','value':'Waarde Bitcoin'})
fig.update_layout(title_text='Grafiek Bitcoin analyse', font_size=15, font_color='black',legend_title_text='Bitcoin Parameters:')
fig.for_each_trace(lambda t:  t.update(name = next(names)))
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)

fig.show()

# 2.7 Analyse jaar: 2020

Ik maak een variabele aan met de naam: jaar_2020 die alleen de gegevens van jaar_2020 zal bevatten. De kolommen 'Adj Close' en 'Volume' laat ik weg

In [ ]:
jaar_2020 = maindf.loc[(maindf['Date'] >= '2020')
                     & (maindf['Date'] < '2021')]

jaar_2020.drop(jaar_2020[['Adj Close','Volume']],axis=1)

De gemiddelde waarde per maand weergeven. 

In [ ]:
gemiddelde= jaar_2020.groupby(jaar_2020['Date'].dt.strftime('%B'))[['Open','Close']].mean()
maanden = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 
             'September', 'October', 'November', 'December']
gemiddelde = gemiddelde.reindex(maanden, axis=0)
gemiddelde

Een barchart die laat zien wat de opening prijs en de gesloten prijs is van het gemiddelde van elke maand. Geel: open, Groen: gesloten

In [ ]:
fig = go.Figure()

fig.add_trace(go.Bar(
    x=gemiddelde.index,
    y=gemiddelde['Open'],
    name='Bitcoin ppen prijs',
    marker_color='Yellow'
))
fig.add_trace(go.Bar(
    x=gemiddelde.index,
    y=gemiddelde['Close'],
    name='Bitcoin gesloten prijs',
    marker_color='Green'
))

fig.update_layout(barmode='group', xaxis_tickangle=-45, 
                  title='Maandelijkse vergelijking tussen open en gesloten prijzen')
fig.show()

Een barchart die laat zien wat de hoge prijs en de lage prijs is van het gemiddelde van elke maand

In [ ]:
jaar_2020.groupby(jaar_2020['Date'].dt.strftime('%B'))['Low'].min()
gemiddelde_high = jaar_2020.groupby(maindf['Date'].dt.strftime('%B'))['High'].max()
gemiddelde_high = gemiddelde_high.reindex(maanden, axis=0)

gemiddelde_low = jaar_2020.groupby(jaar_2020['Date'].dt.strftime('%B'))['Low'].min()
gemiddelde_low = gemiddelde_low.reindex(maanden, axis=0)

fig = go.Figure()
fig.add_trace(go.Bar(
    x=gemiddelde_high.index,
    y=gemiddelde_high,
    name='Bitcoin hoog prijs',
    marker_color='Blue'
))
fig.add_trace(go.Bar(
    x=gemiddelde_low.index,
    y=gemiddelde_low,
    name='Bitcoin laag prijs',
    marker_color='Red'
))

fig.update_layout(barmode='group', 
                  title='Maandelijkse vergelijking tussen Hoge en Lage prijzen')
fig.show()

Een boxplot die de maandelijkse gesloten prijs laat zien van het jaar 2020.

In [ ]:
fig = px.box(jaar_2020,
             x="Maand",
             y="Close",
             labels = {"Close" : "Gesloten prijs"}, 
             title="Maandelijkse vergelijking gesloten prijs")
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

Een boxplot die de gesloten prijs voor elke dag van elke maand laat zien van het jaar 2020.
Dus van alle 30 of 31 dagen de gesloten prijs

In [ ]:
fig = px.box(jaar_2020,
             x="Dag",
             y="Close",
             labels = {"Close" : "Gesloten prijs"}, 
             title="Gesloten prijs voor elke dag van 12 maanden")
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

Een strip plot die de gesloten prijs voor elke dag van van elke maand laat zien van het jaar 2020.

In [ ]:
fig = px.strip(jaar_2020,
               x="Dag",
               y="Close",
               labels = {"Close" : "Gesloten prijs"},
               title = "Gesloten prijs voor elke dag van 12 maanden")
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

Een strip plot die de gesloten prijs voor elke dag van van elke maand laat zien van het jaar 2020. Als extra functie kan je maanden apart bekijken en vergelijken.


In [ ]:
fig = px.strip(jaar_2020,
               x="Dag",
               y="Close",
               labels = {"Close" : "Gesloten prijs"},
               color="Maand",
               title = "Gesloten prijs voor elke dag van 12 maanden"
              )
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

Een line chart die een samenvatting van open, gelosten, hoog en laag laat zien wat de algemene trend van het jaar 2020 was

In [ ]:
names = cycle(['Bitcoin open prijs','Bitcoin gesloten prijs','Bitcoin hoog prijs','Bitcoin laag prijs'])

fig = px.line(jaar_2020, x=jaar_2020.Date, y=[jaar_2020['Open'], jaar_2020['Close'], 
                                          jaar_2020['High'], jaar_2020['Low']],
             labels={'Date': 'Datum','value':'Bitcoin aandeel'})
fig.update_layout(title_text='Grafiek Bitcoin analyse', font_size=15, font_color='black',legend_title_text='Bitcoin Parameters:')
fig.for_each_trace(lambda t:  t.update(name = next(names)))
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)

fig.show()

# 2.8 Analyse jaar: 2021

Ik maak een variabele aan met de naam: jaar_2021 die alleen de gegevens van jaar_2021 zal bevatten. De kolommen 'Adj Close' en 'Volume' laat ik weg

In [ ]:
jaar_2021 = maindf.loc[(maindf['Date'] >= '2021')
                     & (maindf['Date'] < '2022')]

jaar_2021.drop(jaar_2021[['Adj Close','Volume']],axis=1)

De gemiddelde waarde per maand weergeven. 

In [ ]:
gemiddelde= jaar_2021.groupby(jaar_2021['Date'].dt.strftime('%B'))[['Open','Close']].mean()
maanden = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 
             'September', 'October', 'November', 'December']
gemiddelde = gemiddelde.reindex(maanden, axis=0)
gemiddelde

Een barchart die laat zien wat de opening prijs en de gesloten prijs is van het gemiddelde van elke maand. Geel: open, Groen: gesloten

In [ ]:
fig = go.Figure()

fig.add_trace(go.Bar(
    x=gemiddelde.index,
    y=gemiddelde['Open'],
    name='Bitcoin open prijs',
    marker_color='Yellow'
))
fig.add_trace(go.Bar(
    x=gemiddelde.index,
    y=gemiddelde['Close'],
    name='Bitcoin gesloten prijs',
    marker_color='Green'
))

fig.update_layout(barmode='group', xaxis_tickangle=-45, 
                  title='Maandelijkse vergelijking tussen open en gesloten prijzen')
fig.show()

Een barchart die laat zien wat de hoge prijs en de lage prijs is van het gemiddelde van elke maand

In [ ]:
jaar_2021.groupby(jaar_2021['Date'].dt.strftime('%B'))['Low'].min()
gemiddelde_high = jaar_2021.groupby(maindf['Date'].dt.strftime('%B'))['High'].max()
gemiddelde_high = gemiddelde_high.reindex(maanden, axis=0)

gemiddelde_low = jaar_2021.groupby(jaar_2021['Date'].dt.strftime('%B'))['Low'].min()
gemiddelde_low = gemiddelde_low.reindex(maanden, axis=0)

fig = go.Figure()
fig.add_trace(go.Bar(
    x=gemiddelde_high.index,
    y=gemiddelde_high,
    name='Bitcoin hoog prijs',
    marker_color='Blue'
))
fig.add_trace(go.Bar(
    x=gemiddelde_low.index,
    y=gemiddelde_low,
    name='Bitcoin laag prijs',
    marker_color='Red'
))

fig.update_layout(barmode='group', 
                  title='Maandelijkse vergelijking tussen Hoge en Lage prijzen')
fig.show()

Een boxplot die de maandelijkse gesloten prijs laat zien van het jaar 2021.

In [ ]:
fig = px.box(jaar_2021,
             x="Maand",
             y="Close",
             labels = {"Close" : "Gesloten prijs"}, 
             title="Maandelijkse vergelijking gesloten prijs")
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

Een boxplot die de gesloten prijs voor elke dag van elke maand laat zien van het jaar 2021.
Dus van alle 30 of 31 dagen de gesloten prijs

In [ ]:
fig = px.box(jaar_2021,
             x="Dag",
             y="Close",
             labels = {"Close" : "Gesloten prijs"}, 
             title="Gesloten prijs voor elke dag van 12 maanden")
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()


Een strip plot die de gesloten prijs voor elke dag van van elke maand laat zien van het jaar 2021.

In [ ]:
fig = px.strip(jaar_2021,
               x="Dag",
               y="Close",
               labels = {"Close" : "Gesloten prijs"},
               title = "Gesloten prijs voor elke dag van 12 maanden")
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

Een strip plot die de gesloten prijs voor elke dag van van elke maand laat zien van het jaar 2021. Als extra functie kan je maanden apart bekijken en vergelijken.


In [ ]:
fig = px.strip(jaar_2021,
               x="Dag",
               y="Close",
               labels = {"Close" : "Gesloten prijs"},
               color="Maand",
               title = "Gesloten prijs voor elke dag van 12 maanden"
              )
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

Een line chart die een samenvatting van open, gelosten, hoog en laag laat zien wat de algemene trend van het jaar 2021 was

In [ ]:
names = cycle(['Aandeel Open Prijs','Aandeel Gesloten prijs','Aandeel Hoog Prijs','Aandeel Laag Prijs'])

fig = px.line(jaar_2021, x=jaar_2021.Date, y=[jaar_2021['Open'], jaar_2021['Close'], 
                                          jaar_2021['High'], jaar_2021['Low']],
             labels={'Date': 'Datum','value':'Bitcoin aandeel'})
fig.update_layout(title_text='Grafiek Bitcoin analyse', font_size=15, font_color='black',legend_title_text='Bitcoin Parameters:')
fig.for_each_trace(lambda t:  t.update(name = next(names)))
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)

fig.show()

# 2.9 Analyse jaar: 2022 (niet compleet)

Ik maak een variabele aan met de naam: jaar_2022 die alleen de gegevens van jaar_2022 zal bevatten. De kolommen 'Adj Close' en 'Volume' laat ik weg

In [ ]:
jaar_2022 = maindf.loc[(maindf['Date'] >= '2022')
                     & (maindf['Date'] < '2023')]

jaar_2022.drop(jaar_2022[['Adj Close','Volume']],axis=1)

De gemiddelde waarde per maand weergeven. 

In [ ]:
gemiddelde= jaar_2022.groupby(jaar_2022['Date'].dt.strftime('%B'))[['Open','Close']].mean()
maanden = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 
             'September', 'October', 'November', 'December']
gemiddelde = gemiddelde.reindex(maanden, axis=0)
gemiddelde

Een barchart die laat zien wat de opening prijs en de gesloten prijs is van het gemiddelde van elke maand. Geel: open, Groen: gesloten

In [ ]:
fig = go.Figure()

fig.add_trace(go.Bar(
    x=gemiddelde.index,
    y=gemiddelde['Open'],
    name='Bitcoin open prijs',
    marker_color='Yellow'
))
fig.add_trace(go.Bar(
    x=gemiddelde.index,
    y=gemiddelde['Close'],
    name='Bitcoin gesloten prijs',
    marker_color='Green'
))

fig.update_layout(barmode='group', xaxis_tickangle=-45, 
                  title='Maandelijkse vergelijking tussen open en gesloten prijzen')
fig.show()

Een barchart die laat zien wat de hoge prijs en de lage prijs is van het gemiddelde van elke maand

In [ ]:
jaar_2022.groupby(jaar_2022['Date'].dt.strftime('%B'))['Low'].min()
gemiddelde_high = jaar_2022.groupby(maindf['Date'].dt.strftime('%B'))['High'].max()
gemiddelde_high = gemiddelde_high.reindex(maanden, axis=0)

gemiddelde_low = jaar_2022.groupby(jaar_2022['Date'].dt.strftime('%B'))['Low'].min()
gemiddelde_low = gemiddelde_low.reindex(maanden, axis=0)

fig = go.Figure()
fig.add_trace(go.Bar(
    x=gemiddelde_high.index,
    y=gemiddelde_high,
    name='Bitcoin hoog prijs',
    marker_color='Blue'
))
fig.add_trace(go.Bar(
    x=gemiddelde_low.index,
    y=gemiddelde_low,
    name='Bitcoin laag prijs',
    marker_color='Red'
))

fig.update_layout(barmode='group', 
                  title='Maandelijkse vergelijking tussen Hoge en Lage prijzen')
fig.show()

Een boxplot die de maandelijkse gesloten prijs laat zien van het jaar 2022.

In [ ]:
fig = px.box(jaar_2022,
             x="Maand",
             y="Close",
             labels = {"Close" : "Gesloten prijs"}, 
             title="Maandelijkse vergelijking gesloten prijs")
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

Een boxplot die de gesloten prijs voor elke dag van elke maand laat zien van het jaar 2022.
Dus van alle 30 of 31 dagen de gesloten prijs


In [ ]:
fig = px.box(jaar_2022,
             x="Dag",
             y="Close",
             labels = {"Close" : "Gesloten prijs"}, 
             title="Gesloten prijs voor elke dag van 12 maanden")
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

Een strip plot die de gesloten prijs voor elke dag van van elke maand laat zien van het jaar 2022.

In [ ]:
fig = px.strip(jaar_2015,
               x="Dag",
               y="Close",
               labels = {"Close" : "Gesloten prijs"},
               title = "Gesloten prijs voor elke dag van 12 maanden")
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

Een strip plot die de gesloten prijs voor elke dag van van elke maand laat zien van het jaar 2022. Als extra functie kan je maanden apart bekijken en vergelijken.

In [ ]:
fig = px.strip(jaar_2022,
               x="Dag",
               y="Close",
               labels = {"Close" : "Gesloten prijs"},
               color="Maand",
               title = "Gesloten prijs voor elke dag van 12 maanden"
              )
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()


Een line chart die een samenvatting van open, gelosten, hoog en laag laat zien wat de algemene trend van het jaar 2022 was

In [ ]:
names = cycle(['Bitcoin open prijs','Bitcoin gesloten prijs','Bitcoin hoog prijs','Bitcoin laag prijs'])

fig = px.line(jaar_2022, x=jaar_2022.Date, y=[jaar_2022['Open'], jaar_2022['Close'], 
                                          jaar_2022['High'], jaar_2022['Low']],
             labels={'Date': 'Datum','value':'Waarde Bitcoin'})
fig.update_layout(title_text='Grafiek Bitcoin analyse', font_size=15, font_color='black',legend_title_text='Bitcoin Parameters:')
fig.for_each_trace(lambda t:  t.update(name = next(names)))
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)

fig.show()

# 2.10 Totaalanalyse van 2014-2022

Ik maak een variabele aan met de naam: totaalanalyse die alleen de gegevens tussen 2014-2022 laat zien. De kolommen 'Adj Close' en 'Volume' laat ik weg

In [ ]:
totaalanalyse = maindf.loc[(maindf['Date'] >= '2014')
                     & (maindf['Date'] <= '2023')]

totaalanalyse.drop(totaalanalyse[['Adj Close','Volume']],axis=1)

De gemiddelde waarde per maand weergeven. 

In [ ]:
gemiddelde= totaalanalyse.groupby(totaalanalyse['Date'].dt.strftime('%B'))[['Open','Close']].mean()
maanden = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 
             'September', 'October', 'November', 'December']
gemiddelde = gemiddelde.reindex(maanden, axis=0)
gemiddelde

Een line chart die een samenvatting van open, gelosten, hoog en laag laat zien wat de algemene trend van jaar 2014 t/m 2022

In [ ]:
names = cycle(['Aandeel Open Prijs','Aandeel Gesloten Prijs','Aandeel Hoog Prijs','Aandeel Laag Prijs'])

fig = px.line(totaalanalyse, x=totaalanalyse.Date, y=[totaalanalyse['Open'], totaalanalyse['Close'], 
                                          totaalanalyse['High'], totaalanalyse['Low']],
             labels={'Date': 'Datum','value':'Aandeel rpijs'})
fig.update_layout(title_text='Grafiek voor aandelen', font_size=15, font_color='black',legend_title_text='Aandeel Parameters:')
fig.for_each_trace(lambda t:  t.update(name = next(names)))
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)

fig.show()

### LSTM model

LSTM uitleg= LSTM is in staat voor long short time memory. LSTM heeft feedbackverbindingen en is in staat om de volledige reeks gegevens te verwerken, met uitzondering van enkele gegevenspunten zoals afbeeldingen.

Vanwege de mogelijkheid om informatie uit het verleden op te slaan is LSTM erg handig bij het voorspellen van aandelenkoersen. Dit komt doordat de voorspelling van een toekomstige aandelenkoers afhankelijk is van de eerdere koersen.

De eerste stap is het voorbereiden van gegevens voor training en testen. 
Voor training data gebruik ik ongeveer 2,5 jaar aan data. Aangezien de Bitcoin prijs drastisch is gedaald van 200 dollar in 2014 tot 15000 dollar in 2018 en 3000 dollar in 2019 (deze waarden zijn ongeveer gelijk), beschouw ik slechts iets meer dan 2 jaar om dit soort schommelingen in de gegevens te voorkomen. Omdat ik de gesloten prijs van de Bitcoin wil voorspellen, kijk ik alleen naar de gegevens van "Gesloten" 

In [ ]:
# Hier zie je een variable. Ik begin eerst met de gesloten prijs 
geslotendf = maindf[['Date','Close']]
print("Vorm van gesloten dataframe:", geslotendf.shape)

In [ ]:
# Ik maak met pyplot een line chart om de gesloten prijs te zien tussen 2014-2022
fig = px.line(geslotendf, x=geslotendf.Date, y=geslotendf.Close,labels={'Date':'Jaar','Close':'Gesloten'})
fig.update_traces(marker_line_width=2, opacity=0.8, marker_line_color='orange')
fig.update_layout(title_text='Gehele periode van de Bitcoin gesloten prijs 2014-2022', plot_bgcolor='white', 
                  font_size=15, font_color='black')
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

In [ ]:
# om een nauwkeurige analyse te maken gebruik ik data van ongeveer 2.5 jaar. Omdat 2014-2020 niet in de trend zit. Zie figuur hierboven
# alle datums boven 19-02-2021 tellen mee
geslotendf = geslotendf[geslotendf['Date'] > '2020-05-01']
gesloten_aandeel = geslotendf.copy()
print("Totale gegevens voor voorspelling: ",geslotendf.shape[0])

In [ ]:
# start datum en einddatum (dubbele check)
geslotendf

In [ ]:
# gesloten prijs van 04-05-2021 t/m 23-11-2022
fig = px.line(geslotendf, x=geslotendf.Date, y=geslotendf.Close,labels={'Date':'Datum','Close':'Gesloten'})
fig.update_traces(marker_line_width=2, opacity=0.8, marker_line_color='orange')
fig.update_layout(title_text='Voorspelde periode om Bitcoin Gesloten koers te voorspellen', 
                  plot_bgcolor='white', font_size=15, font_color='black')
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

## Normaliseren van data 

Normalisatie is een techniek die vaak wordt toegepast als onderdeel van de voorbereiding van gegevens voor machine learning. 
Het doel van normalisatie is om de waarden van numerieke kolommen in de dataset te wijzigen om een gemeenschappelijke schaal te gebruiken, zonder de verschillen in het waardenbereik te verstoren of informatie te verliezen.

MinMaxScaler

Voor elke waarde in een kenmerk trekt MinMaxScaler de minimumwaarde in het kenmerk af en deelt dan door het bereik. Het bereik is het verschil tussen het oorspronkelijke maximum en het oorspronkelijke minimum. MinMaxScaler behoudt de vorm van de oorspronkelijke verdeling.

Het doel van normalisatie is om de waarden van numerieke kolommen in de gegevensverzameling te wijzigen in een gemeenschappelijke schaal, zonder de verschillen in het waardenbereik te verstoren.
Normalisatie zorgt ervoor dat de functies consistenter met elkaar zijn, waardoor het model de output nauwkeuriger kan voorspellen.

In [ ]:
geslotendf

In [ ]:
# Hier verwijder ik de "Date" kolom omdat ik die niet nodig heb.
# Ik maak een variabele scaler aan en normaliseer het met MinMax-scaler. 

del geslotendf['Date']
scaler=MinMaxScaler(feature_range=(0,1))
geslotendf=scaler.fit_transform(np.array(geslotendf).reshape(-1,1))
print(geslotendf.shape)

In [ ]:
# Hier zie je arrays. Zo ziet de genormaliseerde data eruit
geslotendf

### Nu ga ik een verdeling maken van gegevens in een trainingsset en een testset

In [ ]:
# Ik houd de trainingsset op 60% en de testset op 40%.
training_size=int(len(geslotendf)*0.60)
test_size=len(geslotendf)-training_size
train_data,test_data=geslotendf[0:training_size,:],geslotendf[training_size:len(geslotendf),:1]
print("train_data: ", train_data.shape)
print("test_data: ", test_data.shape)

### In deze stap transformeer ik de gesloten koers op basis van de voorspellingsvereisten van de Time-series-analysis. Hier neem ik de timestamp als 15

In [ ]:
# Ik zal de array van waarden in de datamatrix omzetten. Ik maak daarvoor een function die dataset maakt 
# en geef ik de dataset de timestamp door. 
# Hier doe ik een array van waarden omzetten in een dataset matrix 
def create_dataset(dataset, time_step=1):
    # empty list
    dataX, dataY = [], []
    for i in range(len(dataset)-time_step-1):
        a = dataset[i:(i+time_step), 0]   ###i=0, 0,1,2,3-----99   100 
        dataX.append(a)
        dataY.append(dataset[i + time_step, 0])
    return np.array(dataX), np.array(dataY)

In [ ]:
# Timestamp instellen en initiëren. 
# Dataset aanmaken
time_step = 15
X_train, y_train = create_dataset(train_data, time_step)
X_test, y_test = create_dataset(test_data, time_step)

print("X_train: ", X_train.shape)
print("y_train: ", y_train.shape)
print("X_test: ", X_test.shape)
print("y_test", y_test.shape)

In [ ]:
# Input omvormen tot [samples, time steps, features]  die nodig is voor LSTM. 

X_train =X_train.reshape(X_train.shape[0],X_train.shape[1] , 1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1] , 1)

print("X_train: ", X_train.shape)
print("X_test: ", X_test.shape)

### Model bouwen

In [ ]:
# Ik maak een variable aan(model) en initaliseer het met sequential 
# Om mijn input in het algoritme te krijgen converteer ik de input naar een readable format, de input shape

model=Sequential()
 
# Dense Layer: simple layer of neurons in which each neuron receives the input from all the neurons of previous layer


#Softmax: is a mathematical function that converts a vector of numbers into a vector of probabilities, 
#where the probabilities of each value are proportional to the relative scale of each value in the vector.
model.add(LSTM(10,input_shape=(None,1),activation="softmax")) 

model.add(Dense(1))

#Adam is an optimization algorithm that can be used instead of the classical stochastic gradient descent procedure 
#to update network weights iterative based in training data.
model.compile(loss="mean_squared_error",optimizer="adam")

In [ ]:
# Ik maak een variable genaamd history en die slaat de history van het model op.
# Validation data = testing data
# epochs =200 , batch_size=32 en verbose=1(loss and val_loss). 
history = model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=200,batch_size=32,verbose=1)

### Plotting loss vs Validation loss

In [ ]:
# Hier plot ik trainingverlies en validatieverlies 
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(loss))

plt.plot(epochs, loss, 'r', label='Trainingsverlies')
plt.plot(epochs, val_loss, 'b', label='Validatieverlies')
plt.title('Training en validatieverlies')
plt.legend(loc=0)
plt.figure()

plt.show()

In [ ]:
# Laten we de voorspelling doen en de prestatiecijfers controleren

train_predict=model.predict(X_train)
test_predict=model.predict(X_test)
train_predict.shape, test_predict.shape

### Model evaluatie

In [ ]:
# Data terugzetten naar originele vorm voor de evaluatie 

train_predict = scaler.inverse_transform(train_predict)
test_predict = scaler.inverse_transform(test_predict)
original_ytrain = scaler.inverse_transform(y_train.reshape(-1,1)) 
original_ytest = scaler.inverse_transform(y_test.reshape(-1,1)) 

### Evaluatiecijfers RMSE, MSE en MAE

RMSE uitleg= Bij machine learning is het uiterst nuttig om één cijfer te hebben om de prestaties van een model te beoordelen, of het nu gaat om training of testing. Root mean square error is een van de meest gebruikte maatstaven hiervoor.

MAE uitleg= MAE evalueert de absolute afstand van de waarnemingen (de ingangen van de dataset) tot de voorspellingen op een regressie, waarbij het gemiddelde over alle waarnemingen wordt genomen.

Variance Regression Score uitleg= Is een module. De best mogelijke score is 1,0

R-squared= R-squared geeft je een schatting van de relatie tussen bewegingen van een afhankelijke variabele op basis van de bewegingen van een onafhankelijke variabele. Het vertelt je niet of het door jou gekozen model goed of slecht is, en het vertelt je ook niet of de gegevens en voorspellingen bevooroordeeld zijn


In [ ]:
# Evaluatiecijfers RMSE(root-mean-square error) en MAE(Mean absolute error) voor training en testing

print("Train data RMSE: ", math.sqrt(mean_squared_error(original_ytrain,train_predict)))
print("Train data MSE: ", mean_squared_error(original_ytrain,train_predict))
print("Train data MAE: ", mean_absolute_error(original_ytrain,train_predict))
print("-------------------------------------------------------------------------------------")
print("Test data RMSE: ", math.sqrt(mean_squared_error(original_ytest,test_predict)))
print("Test data MSE: ", mean_squared_error(original_ytest,test_predict))
print("Test data MAE: ", mean_absolute_error(original_ytest,test_predict))

### Variance Regression Score

In [ ]:
# Variance regression score voor training en testing
print("Train data (variance regression score):", 
      explained_variance_score(original_ytrain, train_predict))
print("Test data (variance regression score):", 
      explained_variance_score(original_ytest, test_predict))

### R square score voor regressie

In [ ]:
# R square score voor training en test
print("Train data R2 score:", r2_score(original_ytrain, train_predict))
print("Test data R2 score:", r2_score(original_ytest, test_predict))

## Vergelijking van originele Bitcoin gesloten koers en voorspelde gesloten koers 

In [ ]:
# shift train voorspellingen voor plotten

look_back=time_step
trainPredictPlot = np.empty_like(geslotendf)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back:len(train_predict)+look_back, :] = train_predict
print("Train voorspelde data: ", trainPredictPlot.shape)

# shift test voorspellingen voor plotten
testPredictPlot = np.empty_like(geslotendf)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(train_predict)+(look_back*2)+1:len(geslotendf)-1, :] = test_predict
print("Test voorspelde data: ", testPredictPlot.shape)

names = cycle(['Orginele gesloten prijs','Train voorspelling gesloten prijs','Test voorspelling gesloten prijs'])


plotdf = pd.DataFrame({'date': gesloten_aandeel['Date'],
                       'original_close': gesloten_aandeel['Close'],
                      'train_predicted_close': trainPredictPlot.reshape(1,-1)[0].tolist(),
                      'test_predicted_close': testPredictPlot.reshape(1,-1)[0].tolist()})

fig = px.line(plotdf,x=plotdf['date'], y=[plotdf['original_close'],plotdf['train_predicted_close'],
                                          plotdf['test_predicted_close']],
              labels={'value':'Aandeel prijs','date': 'Datum'})
fig.update_layout(title_text='Vergelijking tussen oorspronkelijke gesloten koers vs. voorspelde gesloten koers',
                  plot_bgcolor='white', font_size=15, font_color='black', legend_title_text='Prijzen:')
fig.for_each_trace(lambda t:  t.update(name = next(names)))

fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

# De komende 30 dagen voorspellen van Bitcoin

In [ ]:
# x_input van test_data en lengte

x_input=test_data[len(test_data)-time_step:].reshape(1,-1)
temp_input=list(x_input)
temp_input=temp_input[0].tolist()

from numpy import array
# ik maak een array voor de timestamp
lst_output=[]
n_steps=time_step
i=0
# voorspelling van dagen wordt 30
voorspelling_dagen = 30
while(i<voorspelling_dagen):
    
    if(len(temp_input)>time_step):
        
        x_input=np.array(temp_input[1:])
        #print("{} day input {}".format(i,x_input))
        x_input = x_input.reshape(1,-1)
        x_input = x_input.reshape((1, n_steps, 1))
        
        #input is de sequence die ik heb gemaakt
        yhat = model.predict(x_input, verbose=0)
        #print("{} day output {}".format(i,yhat))
        temp_input.extend(yhat[0].tolist())
        temp_input=temp_input[1:]
        #print(temp_input)
       
        lst_output.extend(yhat.tolist())
        i=i+1
        
    else:
        
        x_input = x_input.reshape((1, n_steps,1))
        yhat = model.predict(x_input, verbose=0)
        temp_input.extend(yhat[0].tolist())
        
        lst_output.extend(yhat.tolist())
        i=i+1
               
print("Output van voorspelde volgende dagen: ", len(lst_output))

# Plotting van laatste 15 dagen van dataset en 30 dagen( voorspelling)

In [ ]:
 #de eerste 15 dagen dagen 1 t/m 15
 #daarna de dagen 16 t/m 45 (30 dagen)
laatste_dagen=np.arange(1,time_step+1)
dagen_voorspelling=np.arange(time_step+1,time_step+voorspelling_dagen+1)
print(laatste_dagen)
print(dagen_voorspelling)
print("-------------------------------------------------------------------------------------")
print (voorspelling_dagen)

In [ ]:
temp_mat = np.empty((len(laatste_dagen)+voorspelling_dagen+1,1))
temp_mat[:] = np.nan
temp_mat = temp_mat.reshape(1,-1).tolist()[0]

laatste_originele_dagen_waarde = temp_mat
volgende_voorspelde_dagen_waarde = temp_mat

laatste_originele_dagen_waarde[0:time_step+1] = scaler.inverse_transform(geslotendf[len(geslotendf)-time_step:]).reshape(1,-1).tolist()[0]
volgende_voorspelde_dagen_waarde[time_step+1:] = scaler.inverse_transform(np.array(lst_output).reshape(-1,1)).reshape(1,-1).tolist()[0]

new_pred_plot = pd.DataFrame({
    'laatste_originele_dagen_waarde':laatste_originele_dagen_waarde,
    'volgende_voorspelde_dagen_waarde':volgende_voorspelde_dagen_waarde
})

names = cycle(['laatste 15 dagen gesloten prijs','voorspelling volgende 30 dagen gesloten prijs'])

fig = px.line(new_pred_plot,x=new_pred_plot.index, y=[new_pred_plot['laatste_originele_dagen_waarde'],
                                                      new_pred_plot['volgende_voorspelde_dagen_waarde']],
              labels={'value': 'Aandeel prijs','index': 'Timestamp'})
fig.update_layout(title_text='Laatste 15 dagen vs voorspelling 30 dagen',
                  plot_bgcolor='white', font_size=15, font_color='black',legend_title_text='Gesloten prijs')

fig.for_each_trace(lambda t:  t.update(name = next(names)))
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()



###  Het plotten van volledige gesloten aandeel prijs met volgende 30 dagen periode van voorspelling

In [ ]:
# Dit is de hele voorspellingsgrafiek die eruit zou zien 
lstmdf=geslotendf.tolist()
lstmdf.extend((np.array(lst_output).reshape(-1,1)).tolist())
lstmdf=scaler.inverse_transform(lstmdf).reshape(1,-1).tolist()[0]

names = cycle(['Gesloten prijs'])

fig = px.line(lstmdf,labels={'value': 'Aandeel prijs','index': 'Timesstap'})
fig.update_layout(title_text='Plotten van gehele gesloten koers met voorspelling',
                  plot_bgcolor='white', font_size=15, font_color='black',legend_title_text='Bitcoin')

fig.for_each_trace(lambda t:  t.update(name = next(names)))

fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

#### Bibliografie voor notebook:

##### Visualisaties:
Inspiratie voor visualisaties (ik heb ze bijna allemaal zelf gemaakt): https://www.cienciadedatos.net/documentos/py41-forecasting-cryptocurrency-bitcoin-machine-learning-python.html

Visualisaties Plotly: https://www.mth548.org/Tools/Plotly/plotly_plot_types.html#Strip-plot


##### Algoritme:
LSTM model (1): https://rie.binadarma.ac.id/file/conference/a-lstm-method-for-bitcoin-price-prediction-a-case-study-yahoo-finance-stock-market-1639027986.pdf

LSTM model (2): https://www.kaggle.com/code/mekhdigakhramanian/bitcoin-price-prediction-for-10-days-lstm-vs-gru

LSTM model (3): https://www.analyticsvidhya.com/blog/2021/05/bitcoin-price-prediction-using-recurrent-neural-networks-and-lstm/

